In [18]:
import pandas as pd
import pickle5 as pickle

selected_stanza_ents = pd.read_csv('../../data/our-wikipedia-corpus/Tokens_From_Question_side/mini_corpus-10T5CpT.csv')
stanza_ents_main = pd.read_pickle(open('../../data/stanza_ents-from_context.pkl','rb'))

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model_vocab = list(tokenizer.vocab.keys())

In [ ]:
from tqdm.auto import tqdm

#Removing those entities that are also in the model vocab since we don't want 2 of the same token embedding.
ent_in_model_vocab = []
for ent in tqdm(stanza_ents_main):
    if ent in model_vocab:
        ent_in_model_vocab.append(ent)

for ent in ent_in_model_vocab:
    stanza_ents_main.remove(ent)

#Removing all occurrences of the selected entities. This is where we want fresh new entities & contexts.
for ent in list(set(selected_stanza_ents.ent.to_list())):
    if ent in stanza_ents_main:
        stanza_ents_main = list(filter((ent).__ne__, stanza_ents_main))

In [ ]:
from collections import Counter
s = Counter(stanza_ents_main)
sorted_ent_counts = dict(sorted(s.items(), key=lambda item: item[1], reverse=True))

In [ ]:
keys_to_remove = []
for key in sorted_ent_counts.keys():
    if len(key) == 1:
        keys_to_remove.append(key)
        
for key in keys_to_remove:
    sorted_ent_counts.pop(key)

In [ ]:
import wikipedia
from collections import defaultdict

no_of_ents_to_select = 100
selected_ents_text_dict = defaultdict(list)

for ent in tqdm(sorted_ent_counts.keys()):
    if len(selected_ents_text_dict.keys()) == no_of_ents_to_select:
        break

    try:
        search_result = wikipedia.search(str(ent), results=1)[0]
    except:
        continue

    try:
        wiki_query = wikipedia.page(search_result, auto_suggest=False)
        selected_ents_text_dict[ent].append(wiki_query.summary)
        selected_ents_text_dict[ent].append(wiki_query.content)
    except:
        continue
        
    if len(selected_ents_text_dict.keys()) % 100 == 0:
        print(f'No. of entities selected so far: {len(selected_ents_text_dict.keys())}')

#print(f'Entities that were selected: {selected_ents_text_dict.keys()}') 

In [ ]:
sample_id = []
title = []
context = []
question = []
answers = []

for idx, (ent, page_data) in enumerate(selected_ents_text_dict.items()):
    sample_id.append(str(idx))
    title.append(ent)
    context.append(page_data[1])
    question.append(f'What is {ent}?')
    answers.append({'text': [page_data[0]], 'answer_start': [page_data[1].find(page_data[0])]})
    
#Saving the dataframe as parquet since it was messing up formats.
pd.DataFrame(zip(sample_id, title, context, question, answers), 
             columns = ['id', 'title', 'context', 'question'
                       ,'answers']).to_parquet('mini-corpus-for-extended-QA')

In [1]:
from datasets import load_dataset
import os

s = load_dataset('parquet', data_files=os.path.abspath('../../data/our-wikipedia-corpus/mini-corpus-for-extended-QA'))

Using custom data configuration default-d097b25e34eef54d
Found cached dataset parquet (/home/saptarshi/.cache/huggingface/datasets/parquet/default-d097b25e34eef54d/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [3]:
s['train'][0]['context']

'Severe acute respiratory syndrome (SARS) is a viral respiratory disease of zoonotic origin caused by severe acute respiratory syndrome coronavirus (SARS-CoV or SARS-CoV-1), the first identified strain of the SARS coronavirus species severe acute respiratory syndrome–related coronavirus (SARSr-CoV). The first known cases occurred in November 2002, and the syndrome caused the 2002–2004 SARS outbreak. In the 2010s, Chinese scientists traced the virus through the intermediary of Asian palm civets to cave-dwelling horseshoe bats in Xiyang Yi Ethnic Township, Yunnan.SARS was a relatively rare disease; at the end of the epidemic in June 2003, the incidence was 8,469 cases with a case fatality rate (CFR) of 11%. No cases of SARS-CoV-1 have been reported worldwide since 2004.In December 2019, another strain of SARS-CoV was identified as severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2). This new strain causes coronavirus disease 2019 (COVID-19), a disease which brought about the  CO

In [17]:
s['train']['title']

['SARS',
 'DNA',
 'MERS-CoV',
 'the virus',
 'diarrhea',
 'HTNV',
 'H5N1',
 'IFITM5',
 'RSV',
 'E. coli',
 'IFN',
 'PCR',
 'MKP-1',
 'COVID',
 'Spo0A',
 'B cells',
 'respiratory viruses',
 'nucleotide',
 'IFITM3',
 'the study',
 'CC10',
 'MERS',
 'NEAT1',
 'amino acid',
 'our study',
 'influenza virus',
 'GFP',
 'IgG',
 'IAV',
 'the disease',
 'HUVECs',
 'A549 cells',
 'macrophages',
 'influenza viruses',
 'EGR1',
 'SARS-CoV',
 'RIG-I',
 'cytokine',
 'Ebola',
 'CAP',
 'CEACAM1',
 'mucosal',
 'rVSV',
 'Fgl2',
 'C. difficile',
 'ZM',
 'nucleolar',
 'HIV',
 'HMPV',
 'CD40L',
 'Zanamivir',
 'PBS',
 'PEDV',
 'nucleolus',
 'pertussis',
 'consistent with',
 'intracellular',
 'IgM',
 'B cell',
 'coronaviruses',
 'Virus',
 'Human',
 '2019-nCoV',
 'COPD',
 'EBOV',
 'VEEV',
 'L. casei',
 'FKBP11',
 'glycyrrhizin',
 'SIGNR',
 'apoptosis',
 'NEAT1-2',
 'viral RNA',
 'respiratory tract',
 'cysteine',
 'placebo',
 'TNF',
 'neutralizing antibodies',
 'cytokines',
 'IP-10',
 'HBoV1',
 'RNA',
 'infectio

In [5]:
a = 'This is a test sentence where I try to rearrange words'
substring = 'where I try'

In [7]:
start = a.find(substring)
end = start + len(substring)

In [11]:
b = a[0:start] + a[end:]

In [13]:
b

'This is a test sentence  to rearrange words'

In [14]:
sq = load_dataset('squad')

Found cached dataset squad (/home/saptarshi/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453)


  0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
sq['train'][0:100]['context']

['Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building